# Import des librairies nécessaires

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
from sklearn.model_selection import train_test_split


# Sommaire

1. partition du dataset (train, test) --> Done
2. best feature
2. kppv

## Partition

In [3]:
data = pd.read_csv("fetal_health.csv")

data.fetal_health = pd.Categorical(data.fetal_health, categories=[1.0, 2.0, 3.0], ordered=True)

In [7]:
# Création de deux DF (un de labels et un de données)
labels = data.fetal_health
clean_data = data.drop('fetal_health', axis=1)


In [11]:
print(clean_data.shape)
print(labels.shape)

(2126, 21)
(2126,)


Les dimensions sont cohérentes

In [12]:
data_train, data_test, labels_train, labels_test = train_test_split(clean_data, labels, test_size=0.2, random_state=123, shuffle=True, stratify=labels)


Création d'une partition des données avec une proportion 80/20, random state spécifié pour avoir la même partition à chauqe appel, stratify= labels pour conserver les proportions de nos labels 

In [ ]:
labels_train
labels_test